In [1]:
import sys
sys.path.append('..')

from demo.driver import drive
from demo.shared_params import defaults as params

In [2]:
params.steps = 'all'
params.one_time_pad_length = 64
params.key_size = params.one_time_pad_length*8*4
params.num_users = 100
params.num_user_log_per_hour = 1
params.num_patient_zeros = 5
params.basedir='../demo'
params.num_days=20
params.xmax=100

import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
log = drive(params)

2020-04-01 21:55:50 INFO     Removing: ../demo/configs
2020-04-01 21:55:50 INFO     Removing: ../demo/dbs
2020-04-01 21:55:50 INFO     Removing: ../demo/keys
2020-04-01 21:55:50 INFO     setting up config files
2020-04-01 21:55:50 INFO     setting up public/private keys
2020-04-01 21:56:07 INFO     running simulation
2020-04-01 21:56:07 INFO     running simulation for day 0
2020-04-01 21:57:05 INFO     running simulation for day 1
2020-04-01 21:58:16 INFO     running simulation for day 2
2020-04-01 21:59:43 INFO     running simulation for day 3
2020-04-01 22:01:19 INFO     running simulation for day 4
2020-04-01 22:03:09 INFO     running simulation for day 5
2020-04-01 22:05:24 INFO     running simulation for day 6
2020-04-01 22:07:49 INFO     running simulation for day 7
2020-04-01 22:10:30 INFO     running simulation for day 8
2020-04-01 22:14:00 INFO     running simulation for day 9
2020-04-01 22:17:11 INFO     running simulation for day 10
2020-04-01 22:20:58 INFO     running simul

In [4]:
log[0]

{'day': 0,
 'xpos': array([[21, 67, 61, ...,  6, 30, 46],
        [87, 61, 84, ..., 65, 48, 46],
        [30, 34, 83, ..., 97,  7, 43],
        ...,
        [26,  4,  4, ..., 78,  8, 69],
        [15, 24, 37, ..., 45,  3, 15],
        [80, 98, 27, ..., 74, 31, 70]]),
 'ypos': array([[61, 62, 13, ..., 10, 95, 97],
        [93, 65, 77, ..., 18, 83, 55],
        [97, 62, 97, ..., 93, 58, 61],
        ...,
        [42, 77, 66, ..., 48, 64, 67],
        [54, 51, 84, ..., 63, 50, 40],
        [40, 54, 76, ..., 13, 70, 50]]),
 'new_diagnosed_patients': [0, 1, 2, 3, 4],
 'users_notified': [0, 1, 2, 3, 4, 16, 57],
 'new_contacts': defaultdict(list,
             {13: [(1, 16, (54, 7))], 3: [(4, 57, (50, 37))]}),
 'num_results_responsible': 125,
 'num_publishes_for_other_location_auths': 0,
 'num_notifications': 127}

In [5]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import matplotlib.colors as mcolors
tcolors = list(mcolors.TABLEAU_COLORS)
ncol = len(tcolors)
def get_color(i):
    j = i % ncol
    yield tcolors[j]

def get_colors(n):
    return [next(get_color(i)) for i in range(n)]

In [7]:
import matplotlib.animation

num_entries_per_user_per_day = params.num_user_log_per_hour * 24 + 1
num_entries_per_day = num_entries_per_user_per_day * params.num_users 
asize = num_entries_per_user_per_day * num_entries_per_day
fig, ax = plt.subplots()

x, y = np.zeros(num_entries_per_day*params.num_days), np.zeros(num_entries_per_day*params.num_days)
colors = get_colors(params.num_users) * num_entries_per_user_per_day*params.num_days

sc = ax.scatter(x, y, c=colors)

plt.xlim(params.xmin, params.xmax)
plt.ylim(params.xmin, params.xmax)

def animate(i):
    a, b = [], []
    day = int(i / num_entries_per_user_per_day)
    index = i % num_entries_per_user_per_day
    for j in range(params.num_users):
        a += [log[day]['xpos'][j][index]]
        b += [log[day]['ypos'][j][index]]
    sc.set_offsets(np.c_[a, b])
    
ani = matplotlib.animation.FuncAnimation(
    fig, animate, interval=100, repeat=False,
    frames=params.num_days*num_entries_per_user_per_day
)

plt.close()
#HTML(ani.to_html5_video())


In [8]:
import os
movie_file = os.path.join(params.basedir, 'movement.m4v')
ani.save(movie_file, writer='ffmpeg')

2020-04-01 23:13:10 INFO     Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2020-04-01 23:13:10 INFO     MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 432x288 -pix_fmt rgba -r 10.0 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y ../demo/movement.m4v


In [9]:
from IPython.display import Video

Video(movie_file, embed=True)

In [10]:
def rjust_num(val, width, base=10):
    return str(val).rjust(int(width / base + 1))

for i in range(params.num_days):
    for time_slot, contacts in log[i]['new_contacts'].items():
        for contact in contacts:
            msg = 'on day {}, at time {}, carrier {} and person {} contacted at location {}'.format(
                rjust_num(i, params.num_days),
                rjust_num(time_slot, params.xmax),
                contact[0],
                contact[1],
                contact[2]
            )
            print(msg)

on day   0, at time          13, carrier 1 and person 16 contacted at location (54, 7)
on day   0, at time           3, carrier 4 and person 57 contacted at location (50, 37)
on day   1, at time          14, carrier 0 and person 95 contacted at location (18, 60)
on day   1, at time          24, carrier 3 and person 98 contacted at location (9, 57)
on day   1, at time          12, carrier 4 and person 69 contacted at location (63, 76)
on day   3, at time           2, carrier 2 and person 43 contacted at location (59, 55)
on day   4, at time           3, carrier 0 and person 52 contacted at location (88, 7)
on day   5, at time          16, carrier 0 and person 45 contacted at location (98, 45)
on day   5, at time          19, carrier 4 and person 11 contacted at location (78, 59)
on day   5, at time           1, carrier 4 and person 53 contacted at location (96, 86)
on day   6, at time           3, carrier 4 and person 47 contacted at location (40, 60)
on day   7, at time           9, ca

In [11]:
def b2h(x):
    return bytes(x).hex()

In [12]:
import pandas as pd
import sqlite3

In [13]:
def get_df(table, num_rows=5, postfix=''):
    conn = sqlite3.connect("../demo/dbs/{}{}".format(table, postfix))
    limit = ''
    if num_rows is not None:
         limit = 'limit {}'.format(num_rows)
    sql= "select * from {} {};".format(table, limit)
    df = pd.read_sql_query(sql, conn)
    for c in df.head():
        if c.startswith('encrypted'):
            df[c] = df[c].apply(b2h)
    return df 
                                              

In [14]:
get_df('on_device_store', postfix='.0')

,time,person_auth_id,encrypted_otp
0,2019-04-02 21:56:07.512004,1,60a2733307f201fa28defa962644747215538b4fc6dcad...
1,2019-04-02 22:56:07.512004,1,62d648596fc33462b7a042b203fa9dbccdb0cd9769cc5f...
2,2019-04-02 23:56:07.512004,1,64be0abb753097d6418a379458b569a909546df63caeeb...
3,2019-04-03 00:56:07.512004,1,294ae6587eec3bfd36886b8f2bad71a80d240672f8fd40...
4,2019-04-03 01:56:07.512004,1,2542c2698dbce91368ff893f7828fcc9c79ea5ea86e849...


In [15]:
get_df('location_log')

,time,encrypted_location,encrypted_otp
0,2019-04-02 21:56:07.512004,3ffa4491c0c0bc43c93708acd6957a04d8c1f29ae90394...,60a2733307f201fa28defa962644747215538b4fc6dcad...
1,2019-04-02 22:56:07.512004,b4018a9f259ccc7d6e9c80b9b5c486f5b16fd0af37abd6...,62d648596fc33462b7a042b203fa9dbccdb0cd9769cc5f...
2,2019-04-02 23:56:07.512004,2a0e6d37349459a62658475a1a33804ba6202efa4bd8a0...,64be0abb753097d6418a379458b569a909546df63caeeb...
3,2019-04-03 00:56:07.512004,2aa7cd2e72ea8f5bcf5d659663869516e23fe49960e345...,294ae6587eec3bfd36886b8f2bad71a80d240672f8fd40...
4,2019-04-03 01:56:07.512004,97f601e475f8c1398b9d7ff24d300f91cc7c069ad636a1...,2542c2698dbce91368ff893f7828fcc9c79ea5ea86e849...


In [16]:
get_df('contamination_log')

,time,encrypted_location
0,2019-04-02 21:56:07.512004,3ffa4491c0c0bc43c93708acd6957a04d8c1f29ae90394...
1,2019-04-02 22:56:07.512004,b4018a9f259ccc7d6e9c80b9b5c486f5b16fd0af37abd6...
2,2019-04-02 23:56:07.512004,2a0e6d37349459a62658475a1a33804ba6202efa4bd8a0...
3,2019-04-03 00:56:07.512004,2aa7cd2e72ea8f5bcf5d659663869516e23fe49960e345...
4,2019-04-03 01:56:07.512004,97f601e475f8c1398b9d7ff24d300f91cc7c069ad636a1...


In [17]:
get_df('notification_log')

,time,encrypted_otp
0,2019-04-02 21:56:07.512004,60a2733307f201fa28defa962644747215538b4fc6dcad...
1,2019-04-02 22:56:07.512004,62d648596fc33462b7a042b203fa9dbccdb0cd9769cc5f...
2,2019-04-02 23:56:07.512004,64be0abb753097d6418a379458b569a909546df63caeeb...
3,2019-04-03 00:56:07.512004,294ae6587eec3bfd36886b8f2bad71a80d240672f8fd40...
4,2019-04-03 01:56:07.512004,2542c2698dbce91368ff893f7828fcc9c79ea5ea86e849...


In [18]:
notifications = get_df('notification_log', num_rows=None)                       
locations = get_df('location_log', num_rows=None)                               

In [19]:
user_dfs = []
for i in range(params.num_users):
    user_dfs.append(get_df('on_device_store', num_rows=None, postfix='.{}'.format(i)))


In [20]:
import pandas as pd

In [21]:
from lib.crypto import CryptoServer
la_crypto = CryptoServer('../demo/keys/la_auth')

def decrypt_location(location):
    crypt_ba = bytearray.fromhex(location)
    return la_crypto.decrypt(bytes(crypt_ba))

In [22]:
decrypt_location(locations.iloc[0].encrypted_location)

b'21:61'

In [23]:
loc_notif = pd.merge(locations, notifications, on=['encrypted_otp'], how='inner')
loc_notif['decrypted_location'] = loc_notif['encrypted_location'].apply(decrypt_location)

In [24]:
loc_notif[:5]

,time_x,encrypted_location,encrypted_otp,time_y,decrypted_location
0,2019-04-02 21:56:07.512004,3ffa4491c0c0bc43c93708acd6957a04d8c1f29ae90394...,60a2733307f201fa28defa962644747215538b4fc6dcad...,2019-04-02 21:56:07.512004,b'21:61'
1,2019-04-02 22:56:07.512004,b4018a9f259ccc7d6e9c80b9b5c486f5b16fd0af37abd6...,62d648596fc33462b7a042b203fa9dbccdb0cd9769cc5f...,2019-04-02 22:56:07.512004,b'67:62'
2,2019-04-02 23:56:07.512004,2a0e6d37349459a62658475a1a33804ba6202efa4bd8a0...,64be0abb753097d6418a379458b569a909546df63caeeb...,2019-04-02 23:56:07.512004,b'61:13'
3,2019-04-03 00:56:07.512004,2aa7cd2e72ea8f5bcf5d659663869516e23fe49960e345...,294ae6587eec3bfd36886b8f2bad71a80d240672f8fd40...,2019-04-03 00:56:07.512004,b'54:61'
4,2019-04-03 01:56:07.512004,97f601e475f8c1398b9d7ff24d300f91cc7c069ad636a1...,2542c2698dbce91368ff893f7828fcc9c79ea5ea86e849...,2019-04-03 01:56:07.512004,b'42:46'


In [25]:
from collections import defaultdict
user_contact_locations = defaultdict(list)
for i in range(params.num_days):
    for time_slot, contacts in log[i]['new_contacts'].items():
        for contact in contacts:            
            user_contact_locations[contact[0]].append(contact[2])
            user_contact_locations[contact[1]].append(contact[2])

In [26]:
pd.merge(loc_notif, user_dfs[8], on=['encrypted_otp'], how='inner')

,time_x,encrypted_location,encrypted_otp,time_y,decrypted_location,time,person_auth_id
0,2019-04-18 19:56:07.512004,c37e77d9793252d9c6c2e097c46d7e5c4710e7183ac201...,8e76b441a32d3e449c1b7a28f9383f0856c09d7309b75c...,2019-04-18 19:56:07.512004,b'3:99',2019-04-18 19:56:07.512004,1


In [27]:
user_contact_locations

defaultdict(list,
            {1: [(54, 7), (50, 99), (53, 36)],
             16: [(54, 7)],
             4: [(50, 37),
              (63, 76),
              (78, 59),
              (96, 86),
              (40, 60),
              (61, 8),
              (54, 37),
              (69, 27),
              (36, 62),
              (52, 56)],
             57: [(50, 37)],
             0: [(18, 60), (88, 7), (98, 45), (72, 15)],
             95: [(18, 60)],
             3: [(9, 57), (45, 65), (36, 58)],
             98: [(9, 57)],
             69: [(63, 76)],
             2: [(59, 55), (3, 99), (22, 75), (44, 63)],
             43: [(59, 55)],
             52: [(88, 7), (53, 36)],
             45: [(98, 45)],
             11: [(78, 59)],
             53: [(96, 86), (44, 63)],
             47: [(40, 60)],
             37: [(50, 99)],
             94: [(61, 8)],
             12: [(54, 37)],
             35: [(72, 15), (69, 27)],
             9: [(45, 65)],
             10: [(36, 58)],
             

In [28]:
encrypted_logs_and_motion_output_match = True

for i in range(params.num_users):
    if i < params.num_patient_zeros:
        continue
    from_logs = list(pd.merge(loc_notif, user_dfs[i], on=['encrypted_otp'], how='inner')['decrypted_location'])
    from_brownian = user_contact_locations[i]
    from_brownian = ['{}:{}'.format(pos[0], pos[1]).encode() for pos in from_brownian]
    for fl, fb in zip(sorted(from_logs), sorted(from_brownian)):
        if fl != fb:
            print(fl, fb)
            encrypted_logs_and_motion_output_match = False

if encrypted_logs_and_motion_output_match:
    print('Encrypted logs and motion output match')
else:
    print('System failed...')

Encrypted logs and motion output match
